# Day 16

A graph traversal problem with a twist.
Goal here is not minimisation, but maximisation.

Initial thoughts:
1) Look at the input file this time and get a sense of how big the problem is.... it's get 57 valves. So not huge.
2) Might be able to build up a tree of all the options... I'm not sure a classic Dijkstra's algo will work here, as the "value" of visiting a node changes with time.
3) But still wouldn't want to go back to a node after visiting it

# Part 2
Elephants... change:
1. now only have 26 minutes
2. two players can move... 


In [ ]:
testData = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
"""

import re
import pprint
import copy

import multiprocess as mp

# regex primer:
# pattern = '.*x=(-*[0-9]*).*y=(-*[0-9]*).*closest beacon is at x=(-*[0-9]*).*y=(-*[0-9]*)'
# match = re.search(pattern, l)
# sensorPos = (int(match.group(1)),int(match.group(2)))

class Valve:
    def __init__(self, valve, flow, paths):
        self.valve = valve
        self.flow = flow
        self.paths = paths

class Scenario:
    #contains the result and logic for a scenario
    def __init__(self, network):
        self.network = network
        #for a fresh scenario (note: will end up with a lot of scenarios being a deep copy, so they won't reinitalise)
        self.time = 0

        #let's try and make this object as small/simple as possible, so it's quicker to copy
        self.currentValve:str = 'AA'
        self.elephantValve:str = 'AA'
        self.openValveIdentifiers:[str] = []
        self.currentFlowRate:int = 0
        self.cummulativeFlow = 0
        
    def key(self)->str:
        #create a composite key of all the important bit of the scenario, so we can de-dupe
        self.openValveIdentifiers.sort()
        #it doesn't actually make a difference if it's you or elephant, so put positions in order to potentially halve the trouser legs of time
        currentPositions = [self.currentValve, self.elephantValve] #actually, not 100% about that assumption, so let's treat them seperately .sort()
        output = str(self.cummulativeFlow) + '-' + str(currentPositions[0]) + '-' + str(currentPositions[1]) + ' // '
        for valve in self.openValveIdentifiers:
            output = output + valve + ':'
        return output

    def fork(self)->object:
        #rather than use deepcopy all over the place, I want to try some optimisations
        fork = Scenario(self.network)
        fork.time = self.time
        fork.currentValve = self.currentValve
        fork.elephantValve = self.elephantValve
        fork.openValveIdentifiers = self.openValveIdentifiers.copy()
        fork.currentFlowRate = self.currentFlowRate
        fork.cummulativeFlow = self.cummulativeFlow
        return fork

    def clockTick(self)->[]:
        #the scenario tick on a click. Returns multiple realities of scenario. Note: type missing in return because python 3 type system is... weird.
        self.time += 1
        #update the cummulative flow with all open valves
        self.cummulativeFlow += self.currentFlowRate
        output:[] = []
        #open valve or move? Two player mode!
        #You
        currentValveFlow = self.network.valves[self.currentValve].flow
        if not (currentValveFlow == 0 or self.currentValve in self.openValveIdentifiers):
            #opening valve is a valid move
            newScenario = self.fork()
            #You open valve
            newScenario.openValveIdentifiers.append(self.currentValve)
            newScenario.currentFlowRate += currentValveFlow
            output.append(newScenario)
        
        #is is possible that it's better to skip opening a valve an move on? Don't know... so let's include it in an option
        #you move
        for path in self.network.valves[self.currentValve].paths:
            newScenario = self.fork()
            newScenario.currentValve = newScenario.network.valves[path].valve
            output.append(newScenario)

        #now for each of those scenarios, we also need to consider what the elephant is doing.
        output2 = []
        elephantValveFlow = self.network.valves[self.elephantValve].flow
        for sc in output:
            if not (elephantValveFlow == 0 or sc.elephantValve in sc.openValveIdentifiers): #something going wrong here... we're getting duplicated in the open valve list, and I can't work out why.
                #elephant open valves
                newScenario = sc.fork()
                newScenario.openValveIdentifiers.append(sc.elephantValve)    
                newScenario.currentFlowRate += elephantValveFlow
                output2.append(newScenario)
            #moves
            for path in sc.network.valves[sc.elephantValve].paths:
                newScenario = sc.fork()
                newScenario.elephantValve = newScenario.network.valves[path].valve
                output2.append(newScenario)
        return output2



class Network:

    def __init__(self,input:str):
        pattern = 'Valve ([A-Z][A-Z]) has flow rate=([0-9]*); tunnels? leads? to valves? (.*)$'
        self.valves: dict[str,tuple[int,list[str]]] = {} #dictionary of valve -> (flow, [paths])
        for l in input.splitlines():
            match = re.search(pattern,l)
            if match:
                valveName = match.group(1)
                flow = int(match.group(2))
                paths = match.group(3).split(', ')
                valve = Valve(valveName,flow,paths)
                self.valves[valveName] = valve
        #pprint.pprint(self.valves)
    
    def runScenarios(self):
        #simple brute force strategy... whenever there is an option, we fork time.
        forks = {} #this is where we hold all our version of state
        #we always start at minute 1 and run to minute 30
        time = 0
        rootScenario = Scenario(self)
        forks[rootScenario.key()] = rootScenario
        bestScenarioFlow = 0

        while time < 26:
            time += 1
            print('Time t='+str(time)+' Processing forks:'+str(len(forks.keys())))
            #This approach is leading to too many scenarios... we need to cull some as we go.
            #Would the winning scenario always be ahead of the others? Maybe not, but it won't be far behind... but how much.... hmmm... how about 50... try a configurable threshold?
            for scenario in forks.values():
                bestScenarioFlow = max(bestScenarioFlow,scenario.cummulativeFlow)
            threshold = bestScenarioFlow * (9/10) - 20
            print('Best scenario flow='+str(bestScenarioFlow) + '  Setting threshold at:'+str(threshold))
            
            newForks = {}

            
            for scenario in forks.values():
                if scenario.cummulativeFlow >= threshold:
                    for sc in scenario.clockTick():
                        key = sc.key()
                        print(key)
                        newForks[key] = sc
            
            # print('Let\'s hack time... using ' + str(mp.cpu_count())+ ' CPUs')
            # with mp.Pool(mp.cpu_count()) as pool:
            #     for scenario in forks:
            #         if scenario.cummulativeFlow >= threshold:
            #             newForks.extend(pool.apply_async(scenario.clockTick())) #TODO: FIX THIS... it doens't seem to work because apply_asyn is horrible.
            #     pool.join()
            

            forks = newForks

        #we should now have all the scenarios possible in our forks
        print('---')
        print('Total forks: ' + str(len(forks)))
        finalScenarios = list(forks.values())
        finalScenarios.sort(key=lambda x: x.cummulativeFlow, reverse=True)
        winningScenario = finalScenarios[0]
        print('Max Flow: ' + str(winningScenario.cummulativeFlow))
        print(winningScenario.key())





#test data
testnet = Network(testData)
testnet.runScenarios()


Test data result:
Total forks: 10921
Max Flow: 1707
1707-FF-DD // BB:CC:DD:EE:HH:JJ:

Performace: 52.4s

Let's see if we can do better by avoiding the deep copy of the network.
Wow... slashed to 1.2 seconds!




In [ ]:
#run with real input
data = open('day16input.txt').read()
net = Network(data)
net.runScenarios()
